Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

Steps are documented [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-create-your-first-pipeline).

In [29]:
import azureml.core

from azureml.core import Experiment
from azureml.core import Experiment, Workspace, Run, Datastore, Dataset
from azureml.data.data_reference import DataReference
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.core import PipelineData
from azureml.train.sklearn import SKLearn

In [ ]:
ws = Workspace.from_config(path = 'config')

In [ ]:
exp = Experiment(workspace = ws, name = "house_prices_prediction")

In [ ]:
ds = ws.get_default_datastore()

In [ ]:
ds.upload(src_dir = "./data", target_path = "data", overwrite = True, show_progress = True)

In [ ]:
# blob_input_data = DataReference(
#     datastore = def_blob_store,
#     data_reference_name = "input_data",
#     path_on_datastore = "data/ames.csv")

In [35]:
output_data = PipelineData(
    "output_data",
    datastore = def_blob_store,
    output_name = "output_data")

In [ ]:
cluster_name = "compute02"

try:
    compute_target = ComputeTarget(workspace = ws, name = cluster_name)
    print("Found existing compute target")
except ComputeTargetException:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_DS3_V2", 
                                                           max_nodes = 6)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output = True, min_node_count = None, 
                                       timeout_in_minutes = 20)

In [ ]:
%pycat scripts/train_model.py

In [33]:
script_folder = "./scripts"

script_params = {
    "--data-folder": ds.as_mount()
}

train_step = SKLearn(source_directory = script_folder, 
                     compute_target = compute_target,
                     entry_script = "train_model.py",
                     script_params = script_params)

In [34]:
estimator_run = exp.submit(train_step)

In [ ]:
# # list of steps to run
# pipeline_steps = [train_step, score_step]

# from azureml.pipeline.core import Pipeline

# # Build the pipeline
# pipeline = Pipeline(workspace = ws, steps = [pipeline_steps])

In [ ]:
# from azureml.core import Experiment

# # Submit the pipeline to be run
# pipeline_run = exp.submit(pipeline)
# pipeline_run1.wait_for_completion()

In [ ]:
# published_pipeline1 = pipeline_run1.publish_pipeline(
#      name = "My_Published_Pipeline",
#      description = "My Published Pipeline Description",
#      version = "1.0")

In [ ]:
# from azureml.pipeline.core import PublishedPipeline
# import requests

# response = requests.post(published_pipeline1.endpoint,
#                          headers = aad_token,
#                          json = {"ExperimentName": "My_Pipeline",
#                                "ParameterAssignments": {"pipeline_arg": 20}})